The final version of the code was lost, so an early version is provided here.   
There may be some differences and bugs, so please modify it as needed.

In [ ]:
import os
import re
import sys
import math
import time
import json
import math
import torch
import random
import jieba
import shutil
import segeval
import datasets
import numpy as np
import jieba.posseg
import pandas as pd
import jieba.analyse
from torch import nn
from datasets import Dataset
from pandas import DataFrame
from typing import List, Dict
from torch.optim import AdamW
from tqdm.notebook import tqdm
import torch.nn.functional as F
from sklearn.cluster import DBSCAN
from torch.nn.functional import pad
from torch.nn import CrossEntropyLoss
from modelscope.metainfo import Models
from torch.nn.utils.rnn import pad_sequence
from modelscope.models.builder import MODELS
from similarityMetrics import alignmentIndex  # https://github.com/sierra98x/resources
from datasets import load_dataset, load_metric
from typing import List, Optional, Tuple, Union
from transformers.utils import CONFIG_NAME, WEIGHTS_NAME
from transformers import AutoTokenizer, AutoModelForMaskedLM
from modelscope.utils.constant import ConfigFields, ModelFile
from transformers.modeling_outputs import TokenClassifierOutput
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss
from datasets import load_dataset, load_metric, ClassLabel, DatasetDict
from modelscope.models.nlp.ponet import PoNetPreTrainedModel, PoNetModel
from sklearn.metrics import f1_score, classification_report, recall_score
from transformers import (BertModel, BertForTokenClassification, BertForMaskedLM, 
                          VisionEncoderDecoderModel, BertForSequenceClassification, 
                          BertTokenizerFast, BertConfig, PretrainedConfig)

# Replace here
%run './1d_iou.ipynb'
%run './models.ipynb'

device = 'cuda:0'


#### Data

In [ ]:
TS_iou_dice_fct = IOU_1D()

def trans_data(t):
    dev_data_csv = pd.read_csv('./Alimeeting4MUG/'+t+'.csv', sep='\t')
    trans_data = []
    for n in range(len(dev_data_csv)):
        d1 = dict()
        data1 = eval(dev_data_csv['content'][n])
        d1['dialog_id'] = str(n)
        d1['utterance'] = [i['s'] for i in data1['sentences']]
        d1_label = [''] * len(d1['utterance'])
        for x in [i['id']-1 for i in data1['topic_segment_ids']]:
            d1_label[x] = 1.0
        d1['label'] = d1_label
        trans_data.append(d1)
    return trans_data

In [ ]:
id2label = {
    1.0: "B-EOP",
    '': "O",
    0.0: "B-EOP-Invalid"
  }


def read_data_streaming(data_, t='train', window_size=100, batch_size=1, epoch=1):
    data_piece = []
    meeting_length = []
    len_200_sum = 0
    len_200_num = 0
    over_4k_num = 0
    for n in tqdm(range(len(data_))):
        utterance = data_[n]['utterance']
        dialog_id = data_[n]['dialog_id']
        label_eop = [id2label[i] for i in data_[n]['label']]
        assert len(utterance)==len(label_eop)
        label_seg_ids = [-1]+[i for i in range(len(label_eop)) if label_eop[i] != 'O']
        for m in range(1, len(label_seg_ids)-1):
            #block_sents = utterance[label_seg_ids[m-1]+1:label_seg_ids[m]+1]
            #block_label = label_eop[label_seg_ids[m-1]+1:label_seg_ids[m]+1]
            block_sents = utterance[label_seg_ids[m-1]+1:label_seg_ids[m-1]+window_size]
            len_200_sum += len(''.join(block_sents))
            len_200_num += 1
            block_label = label_eop[label_seg_ids[m-1]+1:label_seg_ids[m]+window_size]
            paragraph = [{'timestamp': str(i+1),
                          'speaker': '说话人1',
                          'content': block_sents[i]} for i in range(len(block_sents))]
            inputs = {'dialog_id': dialog_id,
                      'paragraph': paragraph,
                      'label':block_label
                     }

            # 分句（aliMUG不分）
            #paragraph_new, newidx2old = dialog_sentence_split(inputs)
            paragraph_new = {'dialog_id':inputs['dialog_id'], 
                             'paragraph_segment_ids':[{'id': i+1} for i in range(len(inputs['paragraph']))], 
                             'sentences':[{'id': i+1, 'speaker': inputs['paragraph'][i]['speaker'], 
                                           's': inputs['paragraph'][i]['content']} for i in range(len(inputs['paragraph']))], 
                             'topic_segment_ids':[]}
            pred_raw_dataset = Dataset.from_list([{"idx": "0", "content": paragraph_new}])
            pred_eda_one = data_parse_fn(pred_raw_dataset)
            predict_dataset = prepare_input_features(pred_eda_one)
            # 超过4096
            if len(predict_dataset['input_ids'])>1:
            #    print(n,m,len(predict_dataset['input_ids']),predict_dataset['input_ids'][-1].index(0),'Over 4096')
                over_4k_num += 1
            if 0 in predict_dataset['input_ids'][0]:
                no_pad_l = predict_dataset['input_ids'][0].index(0)
                streaming_input_ids = predict_dataset['input_ids'][0][:no_pad_l]
                streaming_attention_mask = predict_dataset['attention_mask'][0][:no_pad_l]
                streaming_segment_ids = predict_dataset['segment_ids'][0][:no_pad_l]
            else:
                streaming_input_ids = predict_dataset['input_ids'][0]
                streaming_attention_mask = predict_dataset['attention_mask'][0]
                streaming_segment_ids = predict_dataset['segment_ids'][0]
            type_id = 0
            token_type_ids_seg_sent = []
            for i in streaming_input_ids:
                token_type_ids_seg_sent.append(type_id)
                if i == 21128:
                    type_id = 1 - type_id
            # 2cls 000...1111...
            new_label = [1 if i!=21128 else 0 for i in streaming_input_ids]
            seg_1_sent_index = inputs['label'].index("B-EOP") # 这个也算
            num_eop = 0
            for i in range(len(new_label)):
                if new_label[i] == 0:
                    num_eop += 1
                if num_eop == seg_1_sent_index+1:
                    seg_1_token_index = i
                    break
            new_label = [0]*(seg_1_token_index+1) + [1]*(len(new_label)-seg_1_token_index-1)
            # 4096内无分割
            if 1 not in new_label:
                # print(n,m,'no segment')
                # 没有分割点的话seg_1_token_index未定义
                new_label = [0]*len(streaming_input_ids)
            assert len(streaming_input_ids)==len(token_type_ids_seg_sent)
            assert len(streaming_input_ids)==len(new_label)
            itasl = [streaming_input_ids,
                     token_type_ids_seg_sent,
                     streaming_attention_mask,
                     streaming_segment_ids,
                     new_label]
            data_piece.append(itasl)
    print(len_200_sum/len_200_num)
    while len(data_piece) % batch_size != 0:
        data_piece.append(data_piece[random.randint(0,len(data_piece)-1)])
    print(t+' length: ', len(data_piece))
    print('Over 4096:', over_4k_num)
    if t == 'train':
        data_piece_epoch = []
        for i in range(epoch):
            random.shuffle(data_piece)
            data_piece_epoch.extend(data_piece)
        data_piece = data_piece_epoch
    data_list = [
        {'input_ids':pad_sequence([torch.tensor(data_piece[j][0]) for j in range(i, i+batch_size)], batch_first=True),
         'token_type_ids':pad_sequence([torch.tensor(data_piece[j][1]) for j in range(i, i+batch_size)], batch_first=True),
         'attention_mask':pad_sequence([torch.tensor(data_piece[j][2]) for j in range(i, i+batch_size)], batch_first=True),
         'segment_ids':pad_sequence([torch.tensor(data_piece[j][3]) for j in range(i, i+batch_size)], batch_first=True),
         'labels':pad_sequence([torch.tensor(data_piece[j][4]) for j in range(i, i+batch_size)], batch_first=True)
        }
        for i in range(0,len(data_piece),batch_size)
    ]
    return data_list


dev_data = read_data_streaming(trans_data(d_t:='dev'), t=d_t, batch_size=1)
#train_data = read_data_streaming(trans_data(d_t:='train'), t=d_t, batch_size=2)

#### Train

In [ ]:
def save_pretrained_model(model, train_loss_avg, eval_loss_avg, num):
    model_path = './model_save/'
    model_name = model_path+'PoNet_'+str(round(train_loss_avg,4))+'_'+str(round(eval_loss_avg,4))+'_'+str(num)
    if not os.path.exists(model_name):
        os.makedirs(model_name)
    model.save_pretrained(model_name)
    torch.save(model.state_dict(), model_name+'/pytorch_model.bin')
    return model_name
    
    
def eval_loss():
    loss_sum = 0
    model.eval()
    with torch.no_grad():
        for n in tqdm(range(len(dev_data))):
            outputs = model(**{k:v.to(device) for k,v in dev_data[n].items()})
            loss = outputs.loss
            loss_sum += float(loss)
    loss_avg = loss_sum/len(dev_data)
    print('Eval loss: {:.8f}'.format(loss_avg))
    return loss_avg
    
    
def train(train_data, lr, eval_steps, do_eval=True, do_test=False):
    print('Training steps: {}'.format(len(train_data)),'-'*20,time.strftime('%Y-%m-%d - %H:%M:%S', time.localtime()))
    train_loss_sum = 0
    save_flag = 0
    trian_loss_list = []
    eval_loss_list = []
    eval_score_list = []
    optimizer = AdamW(model.parameters(), lr=lr)
    model.train()
    for n in tqdm(range(len(train_data))):
        # eval & test
        if n % eval_steps ==0 and n!=0:
            trian_loss_list.append(tls:=(train_loss_sum/(n+1)))
            print('Train loss: {:.8f}'.format(tls))
            if do_eval:
                #dev_loss, dev_seg_invalid_num, dev_iou, dev_f1, dev_pred, dev_label = test(model, dev_data, dev_length_label)
                dev_loss = eval_loss()
                eval_loss_list.append(dev_loss)
                model_save_name = save_pretrained_model(model, trian_loss_list[-1], dev_loss, 1)
            if do_test: pass
            
            model.train()
        optimizer.zero_grad()
        outputs = model(**{k:v.to(device) for k,v in train_data[n].items()})
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        train_loss_sum += float(loss)
    loss_avg = train_loss_sum / len(train_data)
    trian_loss_list.append(loss_avg)
    print('Train loss: {:.8f}'.format(loss_avg))
    dev_loss = eval_loss()
    eval_loss_list.append(dev_loss)
    model_save_name = save_pretrained_model(model, trian_loss_list[-1], dev_loss, 1)
    if do_test: pass
    return trian_loss_list, eval_loss_list

#### Main

In [ ]:
MODEL_DIR = './nlp_ponet_fill-mask_chinese-base' # https://modelscope.cn/models/iic/nlp_ponet_document-segmentation_topic-level_chinese-base/files

tokenizer = BertTokenizerFast.from_pretrained(
    MODEL_DIR,
    use_fast=True,
    use_auth_token=False,
)
tokenizer.padding_side == "right"
tokenizer.add_special_tokens({'eos_token': '[EOS]'})

model = PoNetForTokenClassificationWithIOUloss.from_pretrained(
    model_name_or_path=MODEL_DIR,
    task="token-classification-task", 
    #ignore_mismatched_sizes=True,
    revision="v1.1.0"
    ).to(device)

model.resize_token_embeddings(len(tokenizer))

In [ ]:
lr_l = [1e-4, 5e-5]
batch_size = 2

for lr in lr_l:
    epochs = 20
    print('\nLearning rate: {}'.format(lr))
    train_data = read_data_streaming(trans_data(d_t:='train'), t=d_t, batch_size=batch_size, epoch=epochs)

    e_steps = 1484
    print('Eval steps: ', e_steps)
    
    train_loss_l, dev_loss_l = train(train_data, lr, e_steps, do_eval=True)
    print('done!')

In [ ]:
exit()

#### Test

In [ ]:
def cal_seg_eval(pred, label):
    avg_block_size = round(sum(label)/len(label)/2)
    bs = float(segeval.boundary_similarity(pred, label,n_t=avg_block_size))
    ss = float(segeval.segmentation_similarity(pred, label,n_t=avg_block_size))
    wd = float(segeval.window_diff(pred, label))
    pk = float(segeval.pk(pred, label))
    As = alignmentIndex(pred, label)
    iou = TS_iou_dice_fct.cal_label_pred_iou([sum(label[:x+1])-1 for x in range(len(label))], 
                                             [sum(pred[:x+1])-1 for x in range(len(pred))])
    return {'S':ss, 'B':bs, 'A':As, '1-wd':1-wd, '1-pk':1-pk, 'miou':iou['mean_iou'], 'wiou':iou['weighted_iou']}


def test(window_size=100, stop_block_size=23):
    data_ = trans_data(d_t:='test')
    t=d_t
    batch_size=1
    window_size = window_size
    stop_block_size = stop_block_size

    data_piece = []
    meeting_length = []
    over_4k_num = 0
    test_pred_block_length = []
    test_label_block_length = []
    for n in tqdm(range(len(data_))):
        utterance = data_[n]['utterance']
        dialog_id = data_[n]['dialog_id']
        label_eop = [id2label[i] for i in data_[n]['label']]
        assert len(utterance)==len(label_eop)
        test_label_seg_ids = [-1]+[i for i in range(len(label_eop)) if label_eop[i] != 'O']
        # label block形式
        test_label_block = [test_label_seg_ids[i]-test_label_seg_ids[i-1] for i in range(1,len(test_label_seg_ids))]
        test_label_block_length.append(test_label_block)

        #len(utterance), sum(test_label_block)

        block_length = []
        block_length_merge = []
        while sum(block_length) < len(utterance):
            next_seg_start_id = sum(block_length)
            #print('start:',next_seg_start_id)
            if (len(utterance)-next_seg_start_id)<stop_block_size:
                block_length.append(len(utterance)-next_seg_start_id)
                block_length_merge.append(len(utterance)-next_seg_start_id)
                #print(block_length, sum(block_length),len(utterance))
                break
            #block_sents = utterance[label_seg_ids[m-1]+1:label_seg_ids[m]+1]
            block_sents = utterance[next_seg_start_id: next_seg_start_id+window_size]
            paragraph = [{'timestamp': str(i+1),
                          'speaker': '说话人1',
                          'content': block_sents[i]} for i in range(len(block_sents))]
            inputs = {'dialog_id': dialog_id,
                      'paragraph': paragraph,
                      'label':[]
                     }

            # 分句（ali不分）
            #paragraph_new, newidx2old = dialog_sentence_split(inputs)
            paragraph_new = {'dialog_id':inputs['dialog_id'], 
                             'paragraph_segment_ids':[{'id': i+1} for i in range(len(inputs['paragraph']))], 
                             'sentences':[{'id': i+1, 'speaker': inputs['paragraph'][i]['speaker'], 
                                           's': inputs['paragraph'][i]['content']} for i in range(len(inputs['paragraph']))], 
                             'topic_segment_ids':[]}
            pred_raw_dataset = Dataset.from_list([{"idx": "0", "content": paragraph_new}])
            pred_eda_one = data_parse_fn(pred_raw_dataset)
            predict_dataset = prepare_input_features(pred_eda_one)
            # 超过4096
            if len(predict_dataset['input_ids'])>1:
                #print(n,len(predict_dataset['input_ids']),predict_dataset['input_ids'][-1].index(0),'Over 4096')
                over_4k_num += 1
            if 0 in predict_dataset['input_ids'][0]:
                no_pad_l = predict_dataset['input_ids'][0].index(0)
                streaming_input_ids = predict_dataset['input_ids'][0][:no_pad_l]
                streaming_attention_mask = predict_dataset['attention_mask'][0][:no_pad_l]
                streaming_segment_ids = predict_dataset['segment_ids'][0][:no_pad_l]
            else:
                streaming_input_ids = predict_dataset['input_ids'][0]
                streaming_attention_mask = predict_dataset['attention_mask'][0]
                streaming_segment_ids = predict_dataset['segment_ids'][0]
            type_id = 0
            token_type_ids_seg_sent = []
            for i in streaming_input_ids:
                token_type_ids_seg_sent.append(type_id)
                if i == 21128:
                    type_id = 1 - type_id
            assert len(streaming_input_ids)==len(token_type_ids_seg_sent)
            ## predict
            inputs = {'input_ids':torch.tensor([streaming_input_ids]),
                      'token_type_ids':torch.tensor([token_type_ids_seg_sent]),
                      'attention_mask':torch.tensor([streaming_attention_mask]),
                      'segment_ids':torch.tensor([streaming_segment_ids])}
            with torch.no_grad():
                outputs = model(**{k:v.to(device) for k,v in inputs.items()})
            # 计算分割位置
            pred = torch.argmax(outputs.logits,-1)[0].tolist()
            if 1 not in pred:
                # 全部不分割
                #print('no')
                seg_sents_num = streaming_input_ids.count(21128)
                seg_sents_num_str = str(seg_sents_num)+'+'
            else:
                first_seg = pred.index(1)-1#.tolist().count(1)
                # first_seg: 第一段最后一个token id
                # 按照分割位置计算前方句子数量
                seg_sents_num = streaming_input_ids[:first_seg+1].count(21128)
                seg_sents_num_str = str(seg_sents_num)
            if seg_sents_num == 0:
                #print('no_0', end='')
                seg_sents_num = streaming_input_ids.count(21128)
                seg_sents_num_str = str(seg_sents_num)+'+'
            block_length.append(seg_sents_num)
            block_length_merge.append(seg_sents_num_str)
            #print(seg_sents_num,block_length, sum(block_length))
        #print(block_length)
        #print(block_length_merge)
        # 当前段落没有分割。与下个段落合并
        i, j = 0, 0
        while i < len(block_length_merge)-1: # 最后一个不管
            if '+' not in block_length_merge[i]:
                i+=1
                j+=1
            else:
                block_length[j] += block_length[j+1]
                block_length.pop(j+1)
                i+=2
                j+=1
            if j >= len(block_length)-1:
                break
        #print(block_length)
        test_pred_block_length.append(block_length)
    
    cal_res = []
    for i in range(len(test_pred_block_length)):
        pred = test_pred_block_length[i]
        label = test_label_block_length[i]
        cal_res.append(cal_seg_eval(pred, label))

    res = {k:sum([i[k] for i in cal_res])/len(cal_res) for k,v in cal_res[0].items()}
    res['f1'] = cal_label_pred_f1_block(test_label_block_length, test_pred_block_length)['EOP']['f1']
    print(res)
    return res, test_pred_block_length, test_label_block_length

In [ ]:
for ws in range(40,150,10):
    test_res, test_pred_b, test_label_b = test(window_size=ws, stop_block_size=23)

    cal_res = cal_segeval(test_pred_b, 
                          test_label_b, 
                          tolerate_dist=21, 
                          type='block', 
                          end_process=True, 
                          decimal_places=2)
    print(ws,'\n',cal_res)

